# Themenmodellierung
## Songtexte populärer Musik in Deutschland von 1954 bis 2022


### 01 - Importieren benötigter Module

In [14]:
#Pandas
import pandas as pd

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
from gensim.models import LdaModel


#Spacy
import spacy

#pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

#Numpy
import numpy as np

### 02 - Vorverarbeitung der Daten

In [5]:
#Einlesen der Songtexte
raw_data = pd.read_csv("lyrics_topic_model_ready.csv", delimiter = ';')

In [6]:
def lemmatization(texts, allowed_postags = ['NOUN', 'ADJ', 'VERB']):
    #Die Bibliothek spacy wird genutzt, um Songtexte zu lemmatisieren
    nlp = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])
    #Wörter, welche in Songtexten auftauchen aber keinen inhaltichen Merhwert bieten
    exclude_tokens = ['ooh', 'wit', 'doo', 'gon', 'yah', 'tche', 'lala', 'ouh', 'th', 'owe', 'tamam', 'dee', 'woop', 'ere', 'un', 'haa', 'wha', 'chh', 'wooo', 'wot', 'birch', 'cha', 'dub', 'chooka', 'ahh', 'wop', 'weh', 'wim', 'adio', 'hula', 'samba', 'lana', 'coco', 'de', 'lalala', 'bum', 'waoh', 'th', 'tche', 'wa', 'heja', 'rye', 'dabdudu', 'aha', 'diddy', 'tacata', 're', 'halo', 'skss', 'coo', 'wah', 'aha', 'bodda', 'mamasita', 'hooma', 'bope', 'mamama', 'yea', 'ole', 'sera', 'uhwa', 'neanderthal', 'hay', 'tschau', 'bene', 'wayo', 'quando', 'oho', 'ohoho', 'vie', 'comanchero', 'laa']
    texts_out = []
    
    #Alle Songtexte durchlaufen
    for text in texts:
        #Songtext mit spacy lemmatisieren
        doc = nlp(text)
        new_text = []
        
        #Jedes Wort des lemmatisierten Songtextes durchlaufen
        for token in doc:
            #Wenn das Wort weniger als zwei Buchstabenm hat, ein Stoppwort ist oder keinen inhaltlichen Mehrwert bietet wird es herausgefiltert
            if token.pos_ in allowed_postags and len(token) > 2 and not (token.is_stop or token.lemma_ in exclude_tokens):
                new_text.append(token.lemma_)
        final = " " .join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_data = lemmatization(raw_data['Lyrics'])

In [7]:
#Songtexte werden nocheinmal durch Gensim vorverarbeitet
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_data)

### 03- Aufbau des Textkorpus

In [8]:
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]

### 04 - Aufbau des Modells

Falls bereits ein Themenmodell trainiert worden ist, kann dieser Schritt übersprungen werden. Im Rahmen der Masterarbeit "Die Evolution von Songtexten: Eine NLP-gestützte Sentimentanaylse populärer Musik in Deutschland von 1954 bis 2022" ist bereits ein Modell trainert worden. Dies befindet sich im Verzeichnis "lda_model" und kann geladen werden. Dazu Schritt 04 und Schritt 05 auslassen und direkt zu Schritt 6 springen.

In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6,
                                           random_state=100,
                                           passes=20,
                                           iterations = 100,
                                           alpha='auto',
                                           eta='auto')

### 05 - Speichern des Modells

In [19]:
#Hier wird das trainierte Modell gespeichert
save_path = "lda_model/model"
lda_model.save(save_path)

### 06 - Laden des Modells

In [10]:
#Hier wird ein Modell names "model" aus dem Verzeichnis "lda_model" geladen
#Wurde soeben erst ein Modell trainiert kann dieser Schritt übersprungen werden
save_path = "lda_model/model"
lda_model = LdaModel.load(save_path)

### 07 - Visualisierung des Modells

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30, sort_topics=False)
vis

C:\Users\druep\anaconda3\Lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.041455  0.252230       1        1  11.751452
1     -0.141726 -0.252968       2        1   5.450971
2     -0.168551  0.068336       3        1  22.384782
3      0.127247  0.101873       4        1  36.669660
4      0.221271 -0.073102       5        1   9.656327
5      0.003214 -0.096368       6        1  14.086809, topic_info=       Term         Freq        Total Category  logprob  loglift
19     love  8830.000000  8830.000000  Default  30.0000  30.0000
32     want  3786.000000  3786.000000  Default  29.0000  29.0000
799   dance  1731.000000  1731.000000  Default  28.0000  28.0000
821  little  1701.000000  1701.000000  Default  27.0000  27.0000
95     girl  1872.000000  1872.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
267   thing   283.537705  1539.440326   Topic6  -5.0510   0.2681
16     know   365.679223  6091.466767   Topic6  -4.7966  -0.8530
98     good   289.140819  1805.662936   Topic6  -5.0315   0.1282
269    time   306.096431  3241.983039   Topic6  -4.9745  -0.4001
120     way   253.696724  2436.457062   Topic6  -5.1623  -0.3022

[496 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
349       1  0.156655    air
349       2  0.401745    air
349       3  0.045481    air
349       4  0.108648    air
349       5  0.030320    air
...     ...       ...    ...
34        2  0.107375  young
34        3  0.522692  young
34        4  0.168153  young
34        5  0.014182  young
34        6  0.166127  young

[1207 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6])

In [12]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.045*"dance" + 0.033*"man" + 0.021*"come" + 0.020*"get" + 0.019*"wanna" + 0.019*"let" + 0.013*"music" + 0.012*"beat" + 0.012*"time" + 0.012*"stop"')
(1, '0.055*"little" + 0.042*"girl" + 0.019*"come" + 0.018*"mamy" + 0.013*"blue" + 0.013*"wonderful" + 0.012*"work" + 0.011*"beautiful" + 0.011*"bit" + 0.011*"sweet"')
(2, '0.028*"get" + 0.014*"come" + 0.011*"want" + 0.011*"know" + 0.010*"like" + 0.009*"look" + 0.008*"chance" + 0.007*"man" + 0.007*"say" + 0.007*"good"')
(3, '0.070*"love" + 0.038*"know" + 0.021*"heart" + 0.018*"let" + 0.018*"feel" + 0.017*"time" + 0.016*"tell" + 0.015*"baby" + 0.013*"come" + 0.013*"think"')
(4, '0.057*"want" + 0.032*"baby" + 0.031*"sing" + 0.020*"way" + 0.018*"get" + 0.016*"song" + 0.014*"rock" + 0.013*"come" + 0.010*"fire" + 0.010*"hear"')
(5, '0.022*"come" + 0.020*"night" + 0.014*"tonight" + 0.014*"day" + 0.012*"dream" + 0.011*"go" + 0.011*"beautiful" + 0.011*"say" + 0.010*"star" + 0.010*"happy"')


Beachten: pyLDAvis und gensim Themennummerierung unterscheiden sich um eins.

### 08 - Themen benennen

0: Musik, Tanzen & Feiern  
1: Sonstiges   
2: Gesellschaft & Status  
3: Liebe & Beziehungen      
4: Verlangen & Selbstverwirklichung  
5: Träume & Sehnsüchte  

### 09 - Aktualisierte CSV-Datei erstellen

In [41]:
#Data-Frame "raw_data" wird um eine weitere Spalte ergänzt, welche Themen beinhaltet
topics_list = []
for text in corpus:
    topics = lda_model.get_document_topics(text)
    rounded_topics = [(topic, np.round(prob, decimals=2)) for topic, prob in topics]
    topics_list.append(rounded_topics)
    
raw_data['Topics'] = topics_list

In [42]:
#Hier werden die IDs der Themen durch die Namen der Themen ersetzt 
def replace_topic_indices(topics):
    renamed_topics = [(("Musik, Tanzen & Feiern", value) if index == 0 else ("Sonstiges", value) if index == 1 else ("Gesellschaft & Status", value) if index == 2 else ("Liebe & Beziehungen", value) if index == 3 else ("Verlangen & Selbstverwirklichung", value) if index == 4 else ("Träume & Sehnsüchte", value) if index == 5 else (index, value)) for index, value in topics]
    return renamed_topics

raw_data['Topics'] = raw_data['Topics'].apply(lambda x: replace_topic_indices(x))


In [43]:
#Hier wird eine neue CSV-Datei erstellt, welche die Namen der Themen beinhaltet
raw_data.to_csv('lyrics_including_topics.csv', index=False, sep=';', encoding='utf-8')